In [0]:
import numpy as np
import operator
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def file2matrix(filename):
  file=open(filename)
  n=len(file.readlines())
  return_matrix = np.zeros((n,3))
  class_labels=[]
  file=open(filename)
  index = 0
  for line in file.readlines():
    line = line.strip()
    tokens = line.split('\t')
    return_matrix[index,:]=tokens[0:3]
    class_labels.append(tokens[-1])
    index += 1

  return return_matrix, class_labels

In [0]:
x,y=file2matrix('drive/My Drive/2020winter-MLcamp/content/datingTestSet.txt')
print(x)
print(y)

[[4.0920000e+04 8.3269760e+00 9.5395200e-01]
 [1.4488000e+04 7.1534690e+00 1.6739040e+00]
 [2.6052000e+04 1.4418710e+00 8.0512400e-01]
 ...
 [2.6575000e+04 1.0650102e+01 8.6662700e-01]
 [4.8111000e+04 9.1345280e+00 7.2804500e-01]
 [4.3757000e+04 7.8826010e+00 1.3324460e+00]]
['largeDoses', 'smallDoses', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'largeDoses', 'largeDoses', 'didntLike', 'largeDoses', 'didntLike', 'didntLike', 'smallDoses', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'smallDoses', 'largeDoses', 'smallDoses', 'didntLike', 'smallDoses', 'largeDoses', 'smallDoses', 'largeDoses', 'smallDoses', 'largeDoses', 'smallDoses', 'didntLike', 'largeDoses', 'didntLike', 'largeDoses', 'didntLike', 'smallDoses', 'didntLike', 'didntLike', 'smallDoses', 'largeDoses', 'largeDoses', 'didntLike', 'smallDoses', 'largeDoses', 'largeDoses', 'largeDoses', 'didntLike', 'didntLike', 'didntLike', 'didntLike', 'smallDoses', 'smallDoses', 'didntLike', 'largeDoses', 'smal

In [0]:
#값의 정규 분포를 만드는 코드

def apply_normalizer(dataset, offset, divisor):   
  dataset_normalized = np.zeros(dataset.shape)   #dataset을 행렬의 모양으로 각 값을 0으로 채움
  N = dataset.shape[0]   #dataset의 세로줄을 의미한다
  dataset_normalized = dataset - np.tile(offset, (N,1)) #dataset의 각 값에서 (1,p)dls offset을 (N,1)만큼 곱하여 같은 크기로 만든다 즉dataset크기와 동일
  dataset_normalized = dataset_normalized / np.tile(divisor, (N,1)) #divisor이라는 것을 위와 같이 하여 나눈다 

  return dataset_normalized  #즉 이 것의 최종 리턴 값은 (dataset-offset)/divisor

def normalize_minmax(dataset):
  minval = dataset.min(0)  #0은 세로를 의미 즉 세로의 같은 항목끼리의 min값을 minval변수에 대입한다.
  maxval = dataset.max(0)    #위와 동일 

  dataset_normalized = apply_normalizer(dataset, minval, maxval-minval) 

  return dataset_normalized, minval, maxval-minval 

def normalize_meanstd(dataset):
  meanval = dataset.mean(0)
  stdval = dataset.std(0)


  dataset_normalized = apply_normalizer(dataset, meanval, stdval)   #(dataset-meanval)/stdval = 정규분포

  return dataset_normalized, meanval, stdval  #정규분포, 평균, 표준편차를 return함

In [0]:
def knn(x_test, X_train, labels , k):

  N_train = X_train.shape[0]
# X_train의 모든 데이터 포인트에 대해 x_test로부터 거리를 측정

  diff_mat = np.tile(x_test, (N_train, 1)) - X_train
  # x_test - X_train의 각각의 값 --> 차이를 diff_mat에 저장
  sq_diff_mat = diff_mat ** 2
  # diff_max ^2 --> sq_diff_mat
  sq_distances = sq_diff_mat.sum(axis=1) 
  # column을 축으로 해서 sq_diff_mat을 합산 --> sq_distance
  distances = sq_distances ** 0.5
  # sq_distance에 루트를 씌워줌

# x_test로부터 가장 가까운 k개의 X_train 데이터 포인트를 찾아 
  sorted_dist_indicies = distances.argsort() #거리를 기준으로 줄을 세움
  # distances를 오름차순(작은거부터)으로 정렬해서 -> sorted_dist_dindicies

  class_count = {}
  for i in range(k): #i가 0부터 k-1까지 반복
    vote = labels[sorted_dist_indicies[i]]
    class_count[vote] = class_count.get(vote, 0) + 1 #vote는 3개
  sorted_class_count = sorted(class_count.items(), key=operator.itemgetter(1), reverse=True)
  #sorting을 하면은 가장 큰 값이 앞에 나오니까 그 값을 result에 담아준다.
  result = sorted_class_count[0][0]
  return result

In [0]:
#Read the data test
X, y = file2matrix('drive/My Drive/2020winter-MLcamp/content/datingTestSet.txt')
# X에는 3column의 matrix (마일리지, 게임시간, 아이스크림 소비량)
# y에는 label

holdout_ratio = .2

# EX ) N을 10이라 가정
N = X.shape[0] # 몇줄짜리 데이터인지 -> N
N_ts = int(N*holdout_ratio) # N*0.2
N_tr = N - N_ts # N*0.8

#Split dataset
X_tr = X[0:N_tr,:] # X_tr에는 X의 [0~7,0~3]데이터 저장
y_tr = y[0:N_tr]   # y_tr에는 y의 [0~7]데이터 저장

X_ts = X[N_tr:,:]  # X_ts에는 X의 [8~10,0~3]데이터 저장 
y_ts = y[N_tr:]    # y_ts에는 y의 [8~10]데이터 저장

#Normalize tne dataset
X_normalized_tr, off, div = normalize_minmax(X_tr) 
X_normalized_ts = apply_normalizer(X_ts, off, div) 

#Apply classifier
n_errors = 0 
y_pred_ts = [] 
for i in range(N_ts):
  y_pred_ts.append(knn(X_normalized_ts[i], X_normalized_tr, y_tr, 3))
  if(y_pred_ts[i] != y_ts[i]):
    n_errors += 1
  
print("the accuracy is: %f" % (1 - n_errors/float(N_ts)))
print("tne error rate is: %f" % (n_errors/float(N_ts)))

print("\n---- (Y_true, Y_pred) pairs ----")
print(*list(zip(y_ts, y_pred_ts)), sep = "\n")

the accuracy is: 0.950000
tne error rate is: 0.050000

---- (Y_true, Y_pred) pairs ----
('didntLike', 'didntLike')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('smallDoses', 'smallDoses')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('didntLike', 'smallDoses')
('largeDoses', 'largeDoses')
('smallDoses', 'smallDoses')
('smallDoses', 'smallDoses')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('smallDoses', 'smallDoses')
('largeDoses', 'largeDoses')
('didntLike', 'didntLike')
('didntLike', 'didntLike')
('largeDoses', 'largeDoses')
('largeDoses', 'largeDoses')
('largeDoses', 'largeDoses')
('largeDo